In [1]:
import descriptors
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import KFold, ShuffleSplit, GridSearchCV, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.kernel_ridge import KernelRidge

import os
import glob
from scipy.stats import norm
import math

In [2]:
data = pd.read_excel('search space.xlsx')
data['Mols'] = data['SMILES'].apply(Chem.MolFromSmiles)
data['Mols'] = data['Mols'].apply(Chem.AddHs)

bond_types,x_SOB = descriptors.literal_bag_of_bonds(list(data['Mols']))
x_Estate = descriptors.truncated_Estate_fingerprints(list(data['Mols']))
x_cds = descriptors.custom_descriptor_set(list(data['Mols']))
x_combined_train = np.concatenate((x_SOB[0:88], x_Estate[0:88]), axis=1)
x_combined_test = np.concatenate((x_SOB[88:], x_Estate[88:]), axis=1)

data2 = pd.read_excel('initial dataset.xlsx')
y = data2['HOF(kJ/mol)'].values

In [ ]:
GSmodel = GridSearchCV(SVR(kernel='linear'), cv=5, param_grid={"C": np.logspace(-1, 4, 20), "epsilon": np.logspace(-2, 2, 20)})
GSmodel = GSmodel.fit(x_combined_train, y)
model = GSmodel.best_estimator_

In [ ]:
result=[]
n=1000
for i in range(n):
    bs_sample = np.random.choice(range(len(x_combined_train)), size=len(x_combined_train))
    x_bs = np.array(x_combined_train)[bs_sample]
    y_bs = y[bs_sample]
    model.fit(x_bs,y_bs)
    y_pre = model.predict(x_combined_test)
    result.append(y_pre)

In [ ]:
result = np.asarray(result)
means = np.mean(result,axis=0)
#variance = np.var(result,axis=0)
stds = np.std(result,axis=0)

z = (means-np.max(y))/stds
sdfs = norm.pdf(z)
cdfs = norm.cdf(z)
e = stds*(sdfs + z*cdfs)

e_2 = sorted(e)
itemindex1 = np.where(e == e_2[-1])
print(itemindex1)

In [ ]:
y1 = means[]
print(y1)
mol_smile = data['SMILES'][] # mol_smile index = itemindex1 + 88
print(mol_smile)
mol = Chem.MolFromSmiles(mol_smile)
Draw.MolsToImage([mol],subImgSize=(400, 400))
#df = pd.DataFrame(np.transpose(np.array([mol_smile])),columns=['SMILES'])
#df.to_excel('new_molecualr.xlsx')

In [ ]:
def get_num_atom(mol, atomic_number):
    num = 0
    for atom in mol.GetAtoms():
        atom_num = atom.GetAtomicNum()
        if (atom_num == atomic_number):
            num += 1
    return num
def get_mol_mass(mol):
    mol_mass = 0
    for atom in mol.GetAtoms():
        atom_mass = atom.GetMass()
        mol_mass = mol_mass + atom_mass
    return ('%.2f'%mol_mass)

m = Chem.MolFromSmiles(mol_smile)
m1 = Chem.AddHs(m)
AllChem.EmbedMolecule(m1)
AllChem.MMFFOptimizeMolecule(m1)
AllChem.MolToMolFile(m1,'SVR.lin_Tradeoff_1.mol')
n_C = int(get_num_atom(m1,6))
n_H = int(get_num_atom(m1,1))
n_N = int(get_num_atom(m1,7))
n_O = int(get_num_atom(m1,8))
mass = float(get_mol_mass(m1))
OB = 1600*(n_O - 2*n_C - 0.5*n_H) / mass

print(n_C, n_H, n_N, n_O, mass, OB)